In [3]:
import torch
import torch.nn as nn
import torch
import numpy as np
from torch.optim.optimizer import Optimizer

device = torch.device('cuda')

def gradient(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False):
    '''
    Compute the gradient of `outputs` with respect to `inputs`

    gradient(x.sum(), x)
    gradient((x * y).sum(), [x, y])
    '''
    if torch.is_tensor(inputs):
        inputs = [inputs]
    else:
        inputs = list(inputs)
    grads = torch.autograd.grad(outputs, inputs, grad_outputs,
                                allow_unused=True,
                                retain_graph=retain_graph,
                                create_graph=create_graph)
    grads = [x if x is not None else torch.zeros_like(y) for x, y in zip(grads, inputs)]
    return torch.cat([x.contiguous().view(-1) for x in grads])


def jacobian(outputs, inputs, create_graph=False,retain_graph=True):
    '''
    Compute the Jacobian of `outputs` with respect to `inputs`

    jacobian(x, x)
    jacobian(x * y, [x, y])
    jacobian([x * y, x.sqrt()], [x, y])
    '''
    if torch.is_tensor(outputs):
        outputs = [outputs]
    else:
        outputs = list(outputs)

    if torch.is_tensor(inputs):
        inputs = [inputs]
    else:
        inputs = list(inputs)

    jac = []
    for output in outputs:
        output_flat = output.view(-1)
        output_grad = torch.zeros_like(output_flat)
        for i in range(len(output_flat)):
            output_grad[i] = 1
            jac += [gradient(output_flat, inputs, output_grad, retain_graph, create_graph)]
            output_grad[i] = 0
    return torch.stack(jac)



In [4]:
inputs = torch.randn(10, requires_grad=True)
net = nn.Linear(10, 3)
outputs = net(inputs)
outputs_flat = outputs.view(-1)
outputs_grad = torch.zeros_like(outputs_flat)
jac = []
for i in range(len(outputs_flat)):
    outputs_grad[i] = 1
    jac += [gradient(outputs_flat, inputs, outputs_grad, retain_graph=True, create_graph=False)]
    outputs_grad[i] = 0

torch.stack(jac).shape

torch.Size([3, 10])

### Customized Levenberg-Marquardt Optimizer

In [5]:
class LM(Optimizer):
    '''
    Arguments:
        lr: learning rate (step size) default:1
        alpha: the hyperparameter in the regularization default:0.2
    '''
    def __init__(self, params, lr=1):
        defaults = dict(
            lr = lr
            #alpha = alpha
        )
        super(LM, self).__init__(params, defaults)

        if len(self.param_groups) != 1:
            raise ValueError ("LM doesn't support per-parameter options")    
    
    def step(self, f, J, alpha, closure=None):
        '''
        performs a single step

        '''
        assert len(self.param_groups) == 1
        group = self.param_groups[0]
        lr = group['lr']
        # approximate Hessian
        H = torch.matmul(J.T, J) + torch.eye(J.shape[-1]).to(device) * alpha
        # calculate the update       
        delta_w = -1 * torch.matmul(torch.inverse(H), torch.matmul(J.T, f)).detach()
        offset = 0
        for p in group['params']:
            numel = p.numel()
            p = p + lr * delta_w[offset:offset + numel].view_as(p)
            offset += numel
        

In [6]:
net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 20),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(20, 1),
    )
net.cuda('cuda:0')

optimizer = LM(net.parameters())
BATCH_SIZE = 64
EPOCH = 200
x = torch.unsqueeze(torch.linspace(-10, 10, 1000), dim=1)  
y = torch.sin(x) + 0.2*torch.rand(x.size())
torch_dataset = torch.utils.data.TensorDataset(x, y)

loader = torch.utils.data.DataLoader(dataset=torch_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
prev_loss = float('inf')
alpha = 0.5
for i, (data, target) in enumerate(torch_dataset):
    print (i)
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    out = net(data)
    diff = out - target
    loss = (diff ** 2).item()
    print ('MSE loss')
    print (loss)
    if loss < prev_loss:
        print ('successful iteration')
        if alpha > 1e-5:
            alpha /= 10
    else:
        if alpha < 1e5:
            alpha *= 10
    prev_loss = loss
    J = jacobian(diff, net.parameters())
    optimizer.step(diff, J, alpha)

0
MSE loss
0.07400921732187271
successful iteration
1
MSE loss
0.10278677195310593
2
MSE loss
0.17369677126407623
3
MSE loss
0.08175666630268097
successful iteration
4
MSE loss
0.11438998579978943
5
MSE loss
0.05649910122156143
successful iteration
6
MSE loss
0.03103026933968067
successful iteration
7
MSE loss
0.02678677812218666
successful iteration
8
MSE loss
0.02312987670302391
successful iteration
9
MSE loss
0.01499958522617817
successful iteration
10
MSE loss
0.014151274226605892
successful iteration
11
MSE loss
0.016966229304671288
12
MSE loss
0.028560999780893326
13
MSE loss
0.021453185006976128
successful iteration
14
MSE loss
0.030976083129644394
15
MSE loss
0.020725639536976814
successful iteration
16
MSE loss
4.375731805339456e-06
successful iteration
17
MSE loss
0.00869146827608347
18
MSE loss
0.013908074237406254
19
MSE loss
0.0004175693611614406
successful iteration
20
MSE loss
0.0021089829970151186
21
MSE loss
0.00942202564328909
22
MSE loss
0.004964614752680063
successf

KeyboardInterrupt: 